<a href="https://colab.research.google.com/github/vkrisvasan/100days_OSL/blob/main/10_OSL__Composio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#You have used 0 out of 100 requests in free tier
!pip install groq composio-langchain langchain-groq -q

In [2]:
import os
import getpass
credential_names = ["GROQ_API_KEY"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

Provide your...GROQ_API_KEY··········


In [12]:
from groq import Groq
import json

groq = Groq(api_key=os.environ['GROQ_API_KEY'])
client = groq
MODEL = "llama-3.1-70b-versatile" #'llama3-groq-70b-8192-tool-use-preview'

In [13]:
def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})


In [14]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]

In [17]:
user_prompt = "What is 25 * 4 + 10?"
messages=[
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )


ChatCompletion(id='chatcmpl-09c9b2dc-7b5f-434d-9b9d-c1ab82d29022', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bxyc', function=Function(arguments='{"expression": "25 * 4 + 10"}', name='calculate'), type='function')]))], created=1733970520, model='llama-3.1-70b-versatile', object='chat.completion', system_fingerprint='fp_b6828be2c9', usage=CompletionUsage(completion_tokens=20, prompt_tokens=244, total_tokens=264, completion_time=0.08, prompt_time=0.044969279, queue_time=2.954602831, total_time=0.124969279), x_groq={'id': 'req_01jewb4qs0eddabdjb3px06fnt'})


In [18]:
response_message = response.choices[0].message
print("response message:",response_message)


response message: ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bxyc', function=Function(arguments='{"expression": "25 * 4 + 10"}', name='calculate'), type='function')])


In [24]:
!pip install langchain-groq -q
from langchain_groq import ChatGroq  # Correct import for ChatGroq class
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain, PromptTemplate
from langchain_groq.chat import ChatCompletionMessage, ChatCompletionMessageToolCall, Function
# The chat functionalities might be provided by the 'langchain-groq' package.
# Proceed with the rest of your code using the classes imported from 'langchain_groq.chat'.

ChatCompletionMessage(content=None,
                     role='assistant',
                     function_call=None,
                     tool_calls=[ChatCompletionMessageToolCall(id='call_59xh',
                                 function=Function(arguments='{"expression": "25 * 4 + 10"}', name='calculate'), type='function')])

ModuleNotFoundError: No module named 'langchain_community'

In [20]:
tool_calls = response_message.tool_calls
if tool_calls:
        available_functions = {
            "calculate": calculate,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                expression=function_args.get("expression")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        print(second_response.choices[0].message.content)


25 * 4 = 100
100 + 10 = 110

The result of 25 * 4 + 10 is 110.
